# Analysis of vitus vinifera genes, based on feature selection methods.

In [1]:
import pandas as pd
import numpy as np
import pylab as pl

from IPython.display import clear_output

## Read data

#### Gene espression

In [2]:
df_expressions = pd.read_csv("df_expressions.csv", sep = "\t", skipinitialspace = True)
df_expressions.set_index("LocusTag", inplace = True)
df_expressions.head()

,117,118,119,120,121,122,123,135,136,137,...,2547,2548,2549,2550,2551,2554,2555,2556,2557,2558
LocusTag,,,,,,,,,,,,,,,,,,,,,
VITISV_037663<br>VIT_04s0044g00670<br>VIT_13s0106g00530,0.074122,-0.050508,-0.054257,0.024606,0.235260,-0.072766,0.014740,0.146660,-0.015924,0.643370,...,-0.088366,0.402760,0.020669,-0.343190,0.108590,0.258170,0.061400,-0.088795,0.337030,0.507730
VIT_00s0120g00010,0.251910,0.438920,0.165490,0.170040,0.359180,0.500590,0.387740,0.138600,-0.006880,0.466820,...,-0.832850,-0.364330,-0.447610,0.351690,0.348200,-0.183500,-0.101440,-0.118100,0.088322,0.668720
VIT_00s0120g00030,0.196020,0.208990,0.138000,-0.011565,0.069942,0.140790,0.131210,-0.076353,-0.013316,0.096071,...,-0.030736,0.028623,0.001202,0.036219,0.043602,-0.194420,-0.019498,0.038528,0.014623,-0.053427
VIT_00s0120g00070,0.005119,0.020317,0.015013,0.002750,-0.000942,-0.007854,0.002968,0.004298,-0.023359,-0.039764,...,0.049147,0.041794,0.046453,0.026858,0.024642,0.047360,0.114440,0.024600,-0.022167,0.239030
VIT_00s0120g00080<br>VIT_09s0070g00800<br>VIT_00s0198g00180<br>VIT_00s0173g00030,-0.246480,-0.894880,-0.777630,-0.153840,-0.325990,0.408470,-0.103800,0.026554,-0.267200,0.566730,...,0.218790,0.147860,0.121400,-0.053236,-0.233670,0.016658,-0.006360,0.057350,-0.016801,-0.030682


#### Flavonoidi pathway

In [3]:
df_relations = pd.read_csv("df_relations.csv", sep = "\t", skipinitialspace = True)
df_relations.head()

,Cause,Target
0,VIT_02s0033g00450<br>VIT_02s0033g00380<br>VIT_...,VIT_16s0039g02230
1,VIT_07s0104g00090,VIT_16s0039g02230
2,VIT_07s0005g01210,VIT_03s0017g00710
3,VIT_07s0005g01210,VIT_07s0031g00100
4,VIT_07s0005g01210,VIT_10s0003g02430


#### Compute number of causes for every target

In [4]:
causes_for_target_count = df_relations.groupby("Target")[["Cause"]].count()
causes_for_target_count.head()

,Cause
Target,
VIT_00s0361g00010,3
VIT_00s0361g00020,3
VIT_00s0361g00040,3
VIT_00s0687g00010<br>VIT_00s0521g00010,4
VIT_01s0011g02960,4


## Feature Selection

In [5]:
result_columns = ["Method", "Gene", "Regularization", "DCG", "NumberGenes", 
                                    "NumberPathwayGenes", "Error", "PositionCause", "Weights"]

In [6]:
def DCG(relevances, part = None):
    if part == None:
        relevances = np.asarray(relevances)
    else:
        relevances = np.asarray(relevances)[:part]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.
    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)

def NDCG(relevances, part = None):
    best_dcg = DCG(sorted(relevances, reverse=True), part)
    if best_dcg == 0:
        return 0.
    return DCG(relevances, part) / best_dcg

In [7]:
class GeneSelector:
    def prepare_data_for_prediction(self):
        gene_name = self.gene
        df_droped_gene = df_expressions.drop(gene_name, axis = 0)
        self.x_train = np.array(df_droped_gene).T
        self.y_train = np.array(df_expressions.loc[gene_name, :]).T
        self.index_to_gene = df_droped_gene.index
        self.gene_to_index = dict(zip(df_droped_gene.index, range(self.x_train.shape[1])))
        self.cause_genes = np.where(np.in1d(df_droped_gene.index,
                                df_relations[df_relations["Target"] == gene_name]["Cause"]))[0]
    
    def run(self, filename, genes):
        try:
            self.result = pd.read_csv(filename)
            self.file_has_header = True
            self.f = open(filename, 'a');
        except: 
            self.file_has_header = False
            self.f = open(filename, 'w');        
            self.result = pd.DataFrame(columns = result_columns)
        
        for i_gene, gene in enumerate(genes):
            self.gene = gene
            self.prepare_data_for_prediction()            
            for i_param, param in enumerate(self.get_params()):
                print("Process gene number %d/%d and param  number %d/%d" % (i_gene, len(genes), i_param, len(self.get_params())))
                clear_output(wait = True)
                if np.any(np.logical_and(np.abs(self.result["Regularization"] - param) < 1e-4,
                                         self.result["Gene"] == self.gene)):
                    continue
                if self.is_stop():
                    break
                self.param = param
                self.fit()
                self.selected_features = self.select()
                self.error = self.get_error()
                self.len_all = len(self.selected_features)
                self.len_cause = len(np.intersect1d(self.cause_genes, self.selected_features))
                self.sc = self.get_scores()
                self.order = sorted(self.selected_features, key = lambda sf: self.sc[sf], reverse = True)
                self.dcg = DCG(map(lambda p: 1 if p in self.cause_genes else -0.01, self.order))
                self.position_of_cause = np.where(np.in1d(self.order, self.cause_genes))[0]
                self.weight = ' '.join(map(lambda sf: "%s:%s" % (self.sc[sf], self.index_to_gene[sf]),
                                           self.selected_features))
                self.save()

        self.f.close()
        return self.result
                
    def save(self):
        res = {}
        res["Method"] = self.method
        res["Gene"] = self.gene
        res["Regularization"] = self.param
        res["NumberGenes"] = self.len_all
        res["NumberPathwayGenes"] = self.len_cause
        res["Error"] = self.error    
        res["Weights"] = self.weight
        res["DCG"] = self.dcg
        res["PositionCause"] = " ".join(map(str, self.position_of_cause))
        result_small = pd.DataFrame(res, index = [0])
        result_small.to_csv(self.f, header = not self.file_has_header, index = False)
        self.f.flush()
        self.result = self.result.append(result_small)
        self.file_has_header = True
    
    def is_stop(self):
        return np.any(self.result[self.result["Gene"] == self.gene]["NumberGenes"] > 100)

## Lasso feature selection

In [8]:
from sklearn.linear_model import Lasso
from sklearn.metrics import explained_variance_score

class LassoGeneSelector(GeneSelector):
    def __init__(self):
        self.method = "Lasso"
        
    def fit(self):
        self.reg = Lasso(alpha = self.param)
        self.reg.fit(self.x_train, self.y_train)
        
    def select(self):
        return np.where(self.reg.coef_ > 0)[0]
    
    def get_params(self):
        return np.logspace(start = -1, stop = -2.5, num = 20)
    
    def get_scores(self):
        return np.abs(self.reg.coef_)
    
    def get_error(self):
        return 1 - max(0, explained_variance_score(self.reg.predict(self.x_train), self.y_train))

selector = LassoGeneSelector()

selector.run("lasso_result.csv", causes_for_target_count.index).head()

,DCG,Error,Gene,Method,NumberGenes,NumberPathwayGenes,PositionCause,Regularization,Weights
0,-0.025616,1.0,VIT_00s0361g00010,Lasso,4,0,NaN,0.100000,0.00478981407148:VIT_02s0033g01330 0.000543844...
1,-0.025616,1.0,VIT_00s0361g00010,Lasso,4,0,NaN,0.083378,0.00676755247375:VIT_02s0033g01330 0.001654528...
2,-0.029485,1.0,VIT_00s0361g00010,Lasso,5,0,NaN,0.069519,0.00333691314689:VIT_02s0033g01330 0.004491615...
3,-0.033047,1.0,VIT_00s0361g00010,Lasso,6,0,NaN,0.057964,0.000391457434365:VIT_04s0008g03950 0.00084764...
4,-0.039535,1.0,VIT_00s0361g00010,Lasso,8,0,NaN,0.048329,0.000547869419744:VIT_03s0038g01960 0.00433411...


## RandomizeLasso feature selection

In [11]:
from sklearn.linear_model import RandomizedLasso
from sklearn.linear_model import Lasso
from sklearn.metrics import explained_variance_score

class RandomizedLassoGeneSelector(GeneSelector):
    def __init__(self):
        self.method = "RandomizedLasso"
        
    def fit(self):
        n_resampling = 30
        self.reg = RandomizedLasso(alpha = self.param, n_resampling = n_resampling, n_jobs = 1, 
                          selection_threshold = 0.5/n_resampling, random_state = 2)
        self.reg.fit(self.x_train, self.y_train)
        
    def select(self):
        return self.reg.get_support(indices = True)
    
    def get_params(self):
        return np.logspace(start = -1.5, stop = -3, num = 20)

   
    def get_scores(self):
        return self.reg.scores_
    
    def get_error(self):
        if len(self.selected_features) != 0:
            lasso = Lasso(alpha = self.param)
            x_tr = self.reg.transform(self.x_train)
            lasso.fit(x_tr, self.y_train)   
            return 1 - max(0, explained_variance_score(lasso.predict(x_tr), self.y_train))
        else:
            return 0

selector = RandomizedLassoGeneSelector()
selector.run("randomized_lasso_result.csv", causes_for_target_count.index).head()

,DCG,Error,Gene,Method,NumberGenes,NumberPathwayGenes,PositionCause,Regularization,Weights
0,0.0,0.0,VIT_00s0361g00010,RandomizedLasso,0.0,0.0,,0.031623,
0,0.0,0.0,VIT_00s0361g00010,RandomizedLasso,0.0,0.0,,0.026367,
0,0.0,0.0,VIT_00s0361g00010,RandomizedLasso,0.0,0.0,,0.021984,
0,0.0,0.0,VIT_00s0361g00010,RandomizedLasso,0.0,0.0,,0.018330,
0,0.0,0.0,VIT_00s0361g00010,RandomizedLasso,0.0,0.0,,0.015283,


## RFE feature selection:

In [12]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression

class RandomForestWithCoef(RandomForestRegressor):
    def fit(self, *args, **kwargs):
        super(RandomForestWithCoef, self).fit(*args, **kwargs)
        self.coef_ = self.feature_importances_
        

class RFGeneSelector(GeneSelector):
    def __init__(self):
        self.method = "RFE"
        self.df_precomputed = pd.read_csv("lasso_result.csv")
    
    def prepare_data_for_prediction(self):
        gene_name = self.gene
        df_gene_lasso = self.df_precomputed[self.df_precomputed["Gene"] == self.gene]
        weights = df_gene_lasso.loc[:, "Weights"].iloc[df_gene_lasso.shape[0] - 1]
        genes = map(lambda s: s.split(":")[1], weights.split(" "))
        
        self.x_train = np.array(df_expressions.loc[genes,:]).T
        self.y_train = np.array(df_expressions.loc[gene_name, :]).T
        self.index_to_gene = genes
        self.gene_to_index = dict(zip(self.index_to_gene, range(self.x_train.shape[1])))
        self.cause_genes = np.where(np.in1d(self.index_to_gene,
                                df_relations[df_relations["Target"] == gene_name]["Cause"]))[0]
    
    def fit(self):
        self.reg = RFE(RandomForestWithCoef(n_estimators = 100), n_features_to_select = 1, step = 1)
        self.reg.fit(self.x_train, self.y_train)
        
    def select(self):
        return range(len(self.reg.ranking_))
    
    def get_params(self):
        return [0]

    def is_stop(self):
        return False
    
    def get_scores(self):
        return 1.0/self.reg.ranking_
    
    def get_error(self):
        return 1 - max(0, explained_variance_score(self.reg.predict(self.x_train), self.y_train))
    
selector = RFGeneSelector()
selector.run("rfe_lasso_result.csv", causes_for_target_count.index).head()

,DCG,Error,Gene,Method,NumberGenes,NumberPathwayGenes,PositionCause,Regularization,Weights
0,-0.048230,0.353992,VIT_00s0361g00010,RFE,101.0,1.0,72,0.0,0.0434782608696:VIT_00s0173g00130 0.0116279069...
0,-0.037391,0.154942,VIT_00s0361g00020,RFE,112.0,1.0,38,0.0,0.0909090909091:VIT_00s0145g00110 0.03125:VIT_...
0,0.059067,0.048550,VIT_00s0361g00040,RFE,103.0,1.0,11,0.0,0.0169491525424:VIT_00s0265g00090 0.0149253731...
0,-0.215363,0.120996,VIT_00s0687g00010<br>VIT_00s0521g00010,RFE,104.0,0.0,,0.0,0.0344827586207:VIT_00s0188g00010 0.0243902439...
0,-0.219813,0.045598,VIT_01s0011g02960,RFE,107.0,0.0,,0.0,0.010989010989:VIT_00s0173g00130 0.02222222222...
